In [3]:
import matplotlib.pyplot as plt

# 1. 맥북 전용 폰트 'AppleGothic' 설정
plt.rc('font', family='AppleGothic')

# 2. 그래프 내 마이너스 기호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [5]:
"""
서울시 자치구별 최적 순찰 경로 추천 시스템
NetworkX를 활용한 TSP 기반 순찰 경로 최적화
"""

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from itertools import permutations, product
import pickle
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

# ===== 1. 한글 폰트 설정 =====
def setup_korean_font():
    """
    macOS용 한글 폰트 설정
    """
    try:
        # macOS AppleGothic 폰트 설정
        plt.rcParams['font.family'] = 'AppleGothic'
        plt.rcParams['axes.unicode_minus'] = False
        print("✅ 한글 폰트 설정 완료: AppleGothic")
    except:
        print("⚠️  AppleGothic 폰트를 찾을 수 없습니다. 기본 폰트 사용")
        # 대체 폰트 시도
        try:
            plt.rcParams['font.family'] = 'Malgun Gothic'
        except:
            print("⚠️  한글이 깨질 수 있습니다.")


# ===== 2. 위험 지점 추출 클래스 =====
class RiskPointExtractor:
    """
    특정 자치구 내 위험 지점 추출 및 관리
    """
    
    def __init__(self, model_path='rf_crime_model.pkl', 
                 encoder_path='label_encoders.pkl'):
        """
        모델 및 인코더 로드
        """
        try:
            with open(model_path, 'rb') as f:
                self.model = pickle.load(f)
            with open(encoder_path, 'rb') as f:
                self.encoders = pickle.load(f)
            print(f"✅ 모델 로드 완료: {model_path}")
        except FileNotFoundError:
            print(f"⚠️  모델 파일을 찾을 수 없습니다. 시뮬레이션 모드로 전환합니다.")
            self.model = None
            self.encoders = None
        
        # 시간대 및 장소 카테고리 정의
        self.time_slots = [
            '00시-03시', '03시-06시', '06시-09시', '09시-12시',
            '12시-15시', '15시-18시', '18시-21시', '21시-24시'
        ]
        
        self.location_types = [
            '다중이용시설', '주거지역', '상업지역', '유흥가', 
            '공원', '교통시설', '기타'
        ]
    
    def calculate_risk(self, district, crime_type, time_slot, location_type):
        """
        특정 조건의 위험 확률 계산
        """
        if self.model is None:
            # 시뮬레이션 모드: 가중치 기반 계산
            return self._simulate_risk(district, crime_type, time_slot, location_type)
        
        try:
            # 실제 모델 예측
            features = pd.DataFrame({
                '자치구': [district],
                '범죄유형': [crime_type],
                '시간대': [time_slot],
                '장소': [location_type]
            })
            
            # 레이블 인코딩
            for col in features.columns:
                if col in self.encoders:
                    features[col] = self.encoders[col].transform(features[col])
            
            # 예측
            risk_prob = self.model.predict_proba(features)[0][1]
            return risk_prob
            
        except Exception as e:
            print(f"⚠️  예측 중 오류 발생: {e}")
            return self._simulate_risk(district, crime_type, time_slot, location_type)
    
    def _simulate_risk(self, district, crime_type, time_slot, location_type):
        """
        모델이 없을 때 시뮬레이션 기반 위험도 계산
        """
        # 기본 위험도
        base_risk = 0.25
        
        # 범죄 유형별 가중치
        crime_weights = {
            '폭력': 1.4, '절도': 1.2, '성범죄': 1.5, '마약': 1.3, 
            '사기': 1.1, '기타': 1.0
        }
        
        # 시간대별 가중치
        time_weights = {
            '00시-03시': 1.6, '03시-06시': 1.3, '06시-09시': 0.7,
            '09시-12시': 0.6, '12시-15시': 0.7, '15시-18시': 0.9,
            '18시-21시': 1.2, '21시-24시': 1.5
        }
        
        # 장소별 가중치
        location_weights = {
            '다중이용시설': 1.3, '주거지역': 1.0, '상업지역': 1.2,
            '유흥가': 1.7, '공원': 1.2, '교통시설': 1.1, '기타': 0.9
        }
        
        # 위험도 계산
        risk = base_risk
        risk *= crime_weights.get(crime_type, 1.0)
        risk *= time_weights.get(time_slot, 1.0)
        risk *= location_weights.get(location_type, 1.0)
        
        # 랜덤 변동
        risk *= np.random.uniform(0.85, 1.15)
        
        return min(max(risk, 0.0), 1.0)
    
    def extract_top_risk_points(self, district, crime_type, top_n=5):
        """
        특정 자치구 내 위험 지점 상위 N개 추출
        """
        risk_points = []
        
        # 모든 [시간대 x 장소] 조합에 대해 위험도 계산
        for time_slot in self.time_slots:
            for location_type in self.location_types:
                risk_prob = self.calculate_risk(
                    district, crime_type, time_slot, location_type
                )
                
                risk_points.append({
                    'district': district,
                    'crime_type': crime_type,
                    'time_slot': time_slot,
                    'location_type': location_type,
                    'risk_probability': risk_prob,
                    'risk_percentage': round(risk_prob * 100, 2),
                    'node_id': f"{time_slot}_{location_type}"
                })
        
        # DataFrame으로 변환 및 정렬
        df = pd.DataFrame(risk_points)
        df = df.nlargest(top_n, 'risk_probability')
        
        return df.reset_index(drop=True)

# ===== 3. 순찰 경로 최적화 클래스 =====
class PatrolRouteOptimizer:
    """
    TSP 기반 순찰 경로 최적화
    """
    
    def __init__(self, risk_points_df):
        """
        위험 지점 데이터로 초기화
        """
        self.risk_points = risk_points_df
        self.num_points = len(risk_points_df)
        self.graph = None
        self.distance_matrix = None
    
    def create_distance_matrix(self):
        """
        지점 간 거리 행렬 생성
        (실제 좌표가 없으므로 가상 거리 사용)
        """
        n = self.num_points
        
        # 방법 1: 시간대 차이 기반 거리
        # 같은 시간대면 가깝고, 다른 시간대면 멀다
        time_order = {
            '00시-03시': 0, '03시-06시': 1, '06시-09시': 2, '09시-12시': 3,
            '12시-15시': 4, '15시-18시': 5, '18시-21시': 6, '21시-24시': 7
        }
        
        distance_matrix = np.zeros((n, n))
        
        for i in range(n):
            for j in range(n):
                if i == j:
                    distance_matrix[i][j] = 0
                else:
                    # 시간대 차이
                    time_i = time_order[self.risk_points.iloc[i]['time_slot']]
                    time_j = time_order[self.risk_points.iloc[j]['time_slot']]
                    time_diff = abs(time_i - time_j)
                    
                    # 순환 거리 고려 (24시간 주기)
                    time_diff = min(time_diff, 8 - time_diff)
                    
                    # 장소 유형이 다르면 추가 거리
                    location_penalty = 0
                    if self.risk_points.iloc[i]['location_type'] != \
                       self.risk_points.iloc[j]['location_type']:
                        location_penalty = 2
                    
                    # 최종 거리
                    distance_matrix[i][j] = time_diff * 10 + location_penalty + np.random.uniform(1, 5)
        
        self.distance_matrix = distance_matrix
        return distance_matrix
    
    def solve_tsp_greedy(self):
        """
        탐욕 알고리즘으로 TSP 근사 해결
        (작은 규모에서는 전수 조사도 가능하지만, 확장성을 위해 탐욕 사용)
        """
        n = self.num_points
        
        if n <= 1:
            return list(range(n)), 0
        
        # 시작점: 가장 위험한 지점
        unvisited = set(range(n))
        current = 0  # 첫 번째 지점 (가장 위험)
        route = [current]
        unvisited.remove(current)
        total_distance = 0
        
        # 탐욕적으로 가장 가까운 미방문 지점 선택
        while unvisited:
            nearest = min(unvisited, key=lambda x: self.distance_matrix[current][x])
            total_distance += self.distance_matrix[current][nearest]
            route.append(nearest)
            current = nearest
            unvisited.remove(nearest)
        
        # 시작점으로 돌아오기
        total_distance += self.distance_matrix[route[-1]][route[0]]
        
        return route, total_distance
    
    def solve_tsp_optimal(self):
        """
        완전 탐색으로 최적 경로 찾기 (5개 지점이므로 가능)
        """
        n = self.num_points
        
        if n <= 1:
            return list(range(n)), 0
        
        # 첫 번째 지점을 고정하고 나머지 순열 생성
        best_route = None
        best_distance = float('inf')
        
        # 가능한 모든 경로 시도 (첫 지점 고정)
        for perm in permutations(range(1, n)):
            route = [0] + list(perm)
            
            # 경로 거리 계산
            distance = 0
            for i in range(n - 1):
                distance += self.distance_matrix[route[i]][route[i + 1]]
            # 시작점으로 복귀
            distance += self.distance_matrix[route[-1]][route[0]]
            
            if distance < best_distance:
                best_distance = distance
                best_route = route
        
        return best_route, best_distance
    
    def create_route_graph(self, route):
        """
        NetworkX 그래프 생성
        """
        G = nx.DiGraph()
        
        # 노드 추가
        for idx in range(self.num_points):
            node_data = self.risk_points.iloc[idx]
            G.add_node(
                idx,
                label=f"{node_data['time_slot']}\n{node_data['location_type']}",
                risk=node_data['risk_percentage'],
                time=node_data['time_slot'],
                location=node_data['location_type']
            )
        
        # 엣지 추가 (순찰 경로 순서대로)
        for i in range(len(route)):
            current = route[i]
            next_node = route[(i + 1) % len(route)]
            distance = self.distance_matrix[current][next_node]
            
            G.add_edge(
                current, 
                next_node,
                weight=distance,
                order=i + 1
            )
        
        self.graph = G
        return G
    
    def visualize_route(self, route, output_file='patrol_route_graph.png'):
        """
        순찰 경로 시각화
        """
        G = self.create_route_graph(route)
        
        # 그래프 레이아웃
        pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
        
        # 그림 생성
        plt.figure(figsize=(18, 8))
        
        # 노드 크기 및 색상 (위험도 기반)
        node_colors = [G.nodes[node]['risk'] for node in G.nodes()]
        node_sizes = [G.nodes[node]['risk'] * 30 for node in G.nodes()]
        
        # 노드 그리기
        nx.draw_networkx_nodes(
            G, pos,
            node_color=node_colors,
            node_size=node_sizes,
            cmap='YlOrRd',
            vmin=0, vmax=100,
            alpha=0.9,
            edgecolors='black',
            linewidths=2
        )
        
        # 엣지 그리기 (순찰 경로)
        nx.draw_networkx_edges(
            G, pos,
            edge_color='darkblue',
            width=2,
            alpha=0.7,
            arrows=True,
            arrowsize=20,
            arrowstyle='->',
            connectionstyle='arc3,rad=0.1'
        )
        
        # 엣지 라벨 (순서)
        edge_labels = {(u, v): f"#{d['order']}" 
                      for u, v, d in G.edges(data=True)}
        nx.draw_networkx_edge_labels(
            G, pos,
            edge_labels,
            font_size=10,
            font_weight='bold',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7)
        )
        
        # 노드 라벨
        labels = {node: G.nodes[node]['label'] for node in G.nodes()}
        nx.draw_networkx_labels(
            G, pos,
            labels,
            font_size=9,
            font_weight='bold',
            font_color='black'
        )
        
        # 제목
        district = self.risk_points.iloc[0]['district']
        crime_type = self.risk_points.iloc[0]['crime_type']
        plt.title(
            f'🚔 {district} 최적 순찰 경로 ({crime_type} 범죄 예방)\n'
            f'총 {len(route)}개 위험 지점 | 순찰 순서: {" → ".join([str(r+1) for r in route])} → 1',
            fontsize=14,
            fontweight='bold',
            pad=20
        )
        
        # 범례
        sm = plt.cm.ScalarMappable(
            cmap='YlOrRd',
            norm=plt.Normalize(vmin=0, vmax=100)
        )
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=plt.gca(), shrink=0.8)
        cbar.set_label('위험도 (%)', fontsize=12, fontweight='bold')
        
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        print(f"✅ 순찰 경로 그래프 저장: {output_file}")
        plt.close()

# ===== 4. 리포트 생성 클래스 =====
class PatrolReport:
    """
    순찰 경로 분석 리포트 생성
    """
    
    @staticmethod
    def generate_report(district, crime_type, risk_points_df, route, total_distance):
        """
        BizOps 관점의 분석 리포트 생성
        """
        print("\n" + "=" * 80)
        print(f"🚨 {district} 최적 순찰 경로 분석 리포트")
        print("=" * 80)
        
        # 기본 정보
        print(f"\n📋 분석 정보")
        print(f"   • 자치구: {district}")
        print(f"   • 범죄 유형: {crime_type}")
        print(f"   • 분석 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   • 순찰 지점 수: {len(risk_points_df)}개")
        print(f"   • 총 순찰 거리(가중치): {total_distance:.2f} units")
        
        # 위험 지점 상세
        print(f"\n🎯 위험 지점 상세 (위험도 순)")
        print("─" * 80)
        print(f"{'순위':<6} {'시간대':<15} {'장소 유형':<15} {'위험도':<10} {'우선순위'}")
        print("─" * 80)
        
        for idx, row in risk_points_df.iterrows():
            priority = route.index(idx) + 1 if idx in route else '-'
            print(f"{idx+1:<6} {row['time_slot']:<15} {row['location_type']:<15} "
                  f"{row['risk_percentage']:>6.2f}% {priority:>10}")
        
        # 순찰 경로
        print(f"\n🛣️  최적 순찰 경로 (TSP 최적화 결과)")
        print("─" * 80)
        
        for i, point_idx in enumerate(route, 1):
            row = risk_points_df.iloc[point_idx]
            next_point_idx = route[(i % len(route))]
            distance = 0
            
            if i < len(route):
                # 거리 계산 (실제로는 PatrolRouteOptimizer에서 계산된 값 사용)
                pass
            
            print(f"   {i}순위: {row['time_slot']:>12} | {row['location_type']:<15} "
                  f"| 위험도 {row['risk_percentage']:>6.2f}%")
        
        # 복귀
        print(f"   → 1순위 지점으로 복귀 (순찰 완료)")
        
        # 통계 분석
        print(f"\n📊 위험도 통계")
        print("─" * 80)
        print(f"   • 평균 위험도: {risk_points_df['risk_percentage'].mean():.2f}%")
        print(f"   • 최고 위험도: {risk_points_df['risk_percentage'].max():.2f}%")
        print(f"   • 최저 위험도: {risk_points_df['risk_percentage'].min():.2f}%")
        print(f"   • 위험도 표준편차: {risk_points_df['risk_percentage'].std():.2f}%")
        
        # 시간대 분석
        print(f"\n⏰ 시간대별 분석")
        print("─" * 80)
        time_analysis = risk_points_df.groupby('time_slot')['risk_percentage'].agg(['mean', 'count'])
        for time_slot, data in time_analysis.iterrows():
            print(f"   • {time_slot:>12}: 평균 위험도 {data['mean']:>6.2f}% ({int(data['count'])}개 지점)")
        
        # 장소별 분석
        print(f"\n📍 장소별 분석")
        print("─" * 80)
        location_analysis = risk_points_df.groupby('location_type')['risk_percentage'].agg(['mean', 'count'])
        for location, data in location_analysis.iterrows():
            print(f"   • {location:<15}: 평균 위험도 {data['mean']:>6.2f}% ({int(data['count'])}개 지점)")
        
        # BizOps 제언
        print(f"\n💡 BizOps 제언")
        print("─" * 80)
        
        # 가장 위험한 시간대
        most_dangerous_time = risk_points_df.iloc[0]['time_slot']
        most_dangerous_location = risk_points_df.iloc[0]['location_type']
        highest_risk = risk_points_df.iloc[0]['risk_percentage']
        
        print(f"   1. 집중 순찰 시간대: {most_dangerous_time}")
        print(f"      → 위험도 {highest_risk:.2f}%로 최고 위험 구간")
        print(f"      → 순찰 인력 2배 배치 권장\n")
        
        print(f"   2. 우선 관리 장소: {most_dangerous_location}")
        print(f"      → CCTV 추가 설치 검토")
        print(f"      → 주민 안전 캠페인 강화\n")
        
        print(f"   3. 리소스 최적화")
        print(f"      → 제안 순찰 경로로 {len(route)}개 지점을 효율적으로 커버")
        print(f"      → 순찰 시간 약 {int(total_distance / 10)} ~ {int(total_distance / 10 * 1.5)}분 예상")
        print(f"      → 1일 3회 순찰 시 최대 범죄 예방 효과\n")
        
        print(f"   4. 예산 효율성")
        avg_risk = risk_points_df['risk_percentage'].mean()
        efficiency_score = (avg_risk / len(route)) * 10
        print(f"      → 순찰 효율성 점수: {efficiency_score:.1f}/10")
        print(f"      → 최소 리소스로 고위험 지역 집중 관리 가능")
        
        print("\n" + "=" * 80)

# ===== 5. 메인 실행 함수 =====
def main():
    """
    메인 실행 함수
    """
    setup_korean_font()
    
    print("\n" + "=" * 80)
    print("🚔 서울시 자치구별 최적 순찰 경로 추천 시스템")
    print("=" * 80)
    
    # 파라미터 설정
    district = '마포구'
    crime_type = '폭력'
    top_n = 5
    
    print(f"\n🎯 분석 파라미터")
    print(f"   • 자치구: {district}")
    print(f"   • 범죄 유형: {crime_type}")
    print(f"   • 순찰 지점 수: {top_n}개")
    
    # Step 1: 위험 지점 추출
    print(f"\n⚙️  Step 1: 위험 지점 추출 중...")
    extractor = RiskPointExtractor()
    risk_points = extractor.extract_top_risk_points(district, crime_type, top_n)
    print(f"✅ {len(risk_points)}개 위험 지점 추출 완료")
    
    # Step 2: 최적 경로 계산
    print(f"\n⚙️  Step 2: 최적 순찰 경로 계산 중...")
    optimizer = PatrolRouteOptimizer(risk_points)
    optimizer.create_distance_matrix()
    
    # 최적 경로 찾기 (5개 지점이므로 완전 탐색 가능)
    route, total_distance = optimizer.solve_tsp_optimal()
    print(f"✅ 최적 경로 계산 완료 (총 거리: {total_distance:.2f} units)")
    print(f"   순찰 순서: {' → '.join([str(r+1) for r in route])} → 1")
    
    # Step 3: 시각화
    print(f"\n⚙️  Step 3: 네트워크 그래프 생성 중...")
    optimizer.visualize_route(route)
    
    # Step 4: 리포트 생성
    print(f"\n⚙️  Step 4: 분석 리포트 생성 중...")
    PatrolReport.generate_report(district, crime_type, risk_points, route, total_distance)
    
    print(f"\n✨ 모든 분석이 완료되었습니다!")
    print(f"   📊 그래프: patrol_route_graph.png")
    print("=" * 80 + "\n")

# ===== 6. 다양한 시나리오 분석 함수 =====
def analyze_multiple_scenarios():
    """
    여러 자치구 및 범죄 유형에 대한 일괄 분석
    """
    setup_korean_font()
    
    scenarios = [
        {'district': '마포구', 'crime_type': '폭력'},
        {'district': '강남구', 'crime_type': '절도'},
        {'district': '영등포구', 'crime_type': '성범죄'},
    ]
    
    print("\n" + "=" * 80)
    print("🔍 다중 시나리오 분석")
    print("=" * 80)
    
    for i, scenario in enumerate(scenarios, 1):
        print(f"\n{'─'*80}")
        print(f"시나리오 {i}: {scenario['district']} - {scenario['crime_type']}")
        print(f"{'─'*80}")
        
        # 위험 지점 추출
        extractor = RiskPointExtractor()
        risk_points = extractor.extract_top_risk_points(
            scenario['district'], 
            scenario['crime_type'], 
            5
        )
        
        # 최적 경로 계산
        optimizer = PatrolRouteOptimizer(risk_points)
        optimizer.create_distance_matrix()
        route, total_distance = optimizer.solve_tsp_optimal()
        
        # 시각화
        output_file = f"patrol_route_{scenario['district']}_{scenario['crime_type']}.png"
        optimizer.visualize_route(route, output_file)
        
        # 간단 리포트
        print(f"\n✅ {scenario['district']} 분석 완료")
        print(f"   • 평균 위험도: {risk_points['risk_percentage'].mean():.2f}%")
        print(f"   • 순찰 거리: {total_distance:.2f} units")
        print(f"   • 그래프: {output_file}")

if __name__ == "__main__":
    # 단일 시나리오 분석
    main()
    
    # 다중 시나리오 분석 (선택사항)
    # analyze_multiple_scenarios()

✅ 한글 폰트 설정 완료: AppleGothic

🚔 서울시 자치구별 최적 순찰 경로 추천 시스템

🎯 분석 파라미터
   • 자치구: 마포구
   • 범죄 유형: 폭력
   • 순찰 지점 수: 5개

⚙️  Step 1: 위험 지점 추출 중...
⚠️  모델 파일을 찾을 수 없습니다. 시뮬레이션 모드로 전환합니다.
✅ 5개 위험 지점 추출 완료

⚙️  Step 2: 최적 순찰 경로 계산 중...
✅ 최적 경로 계산 완료 (총 거리: 75.42 units)
   순찰 순서: 1 → 5 → 2 → 4 → 3 → 1

⚙️  Step 3: 네트워크 그래프 생성 중...
✅ 순찰 경로 그래프 저장: patrol_route_graph.png

⚙️  Step 4: 분석 리포트 생성 중...

🚨 마포구 최적 순찰 경로 분석 리포트

📋 분석 정보
   • 자치구: 마포구
   • 범죄 유형: 폭력
   • 분석 일시: 2026-01-23 11:09:30
   • 순찰 지점 수: 5개
   • 총 순찰 거리(가중치): 75.42 units

🎯 위험 지점 상세 (위험도 순)
────────────────────────────────────────────────────────────────────────────────
순위     시간대             장소 유형           위험도        우선순위
────────────────────────────────────────────────────────────────────────────────
1      00시-03시         유흥가             100.00%          1
2      21시-24시         유흥가              90.29%          3
3      03시-06시         유흥가              85.87%          5
4      21시-24시         다중이용시설           77.16%          4
5 